# Data Preparation (CRISP-DM)

Nesta etapa do CRISP-DM, os dados brutos são preparados para análise e
modelagem, incluindo limpeza, padronização, tratamento de valores ausentes
e anonimização de informações sensíveis.

O objetivo é gerar uma base consistente, confiável e reutilizável para
as próximas fases do projeto.

In [19]:
import re
import hashlib
import unicodedata
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

In [20]:
DATA_RAW_PATH = Path("../data/raw")

csv_files = sorted(DATA_RAW_PATH.glob("*.csv"))

dfs = []

for file in csv_files:
    df = pd.read_csv(file, encoding="latin1", sep=";")
    df["arquivo_origem"] = file.name
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df.shape

(24009, 16)

In [21]:
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.replace(r"[^a-z0-9_]", "_", regex=True)
    )

    return df

df.columns

Index(['Referência', 'Nome', 'Cargo', 'Data Admissão', 'Tipo de Regime',
       'Descontos', 'Liquido', 'Data Desligamento', 'Proventos', 'Contrato',
       'Atividade', 'Nome Atividade', 'Tipo de Contrato',
       'Data Prevista Termino Contrato', 'Carga Horária (Sem.)',
       'arquivo_origem'],
      dtype='object')

## Renomeando colunas

In [22]:
# Criando df_prepared como cópia de df_raw
df_prepared = df.copy()

# renomeando colunas
df_prepared.rename(columns={
    "Referência": "referencia",
    "Nome": "nome",
    "Cargo": "cargo",
    "Data Admissão": "data_admissao",
    "Tipo de Regime": "tipo_regime",
    "Descontos": "descontos",
    "Liquido": "liquido",
    "Data Desligamento": "data_desligamento",
    "Proventos": "proventos",
    "Contrato": "contrato",
    "Atividade": "atividade",
    "Nome Atividade": "nome_atividade",
    "Tipo de Contrato": "tipo_contrato",
    "Data Prevista Termino Contrato": "data_prevista_termino_contrato",
    "Carga Horária (Sem.)": "carga_horaria_semanal"
}, inplace=True)

In [23]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,NaN,NaN,Efetivo,NaN,30.0,abr2025.csv
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,abr2025.csv


In [24]:
df_prepared.tail()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv
24008,NaN,NaN,NaN,NaN,NaN,"918.286,26","4.302.011,08",NaN,"5.220.297,34",NaN,NaN,NaN,NaN,NaN,NaN,set2025.csv


## Removendo as linhas de somas finais que vem em cada dataset

In [25]:
df_prepared = df_prepared.dropna(subset=["referencia"])

In [26]:
df_prepared.tail()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,atividade,nome_atividade,tipo_contrato,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem
24003,Folha Mensal - Setembro,YASMIN MICKAELA TAMINI,AGENTE DE DESENVOLVIMENTO INFANTIL,17/01/2025,REGIME PROPRIO,"535,56","2.920,63",NaN,"3.456,19",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24004,Folha Complementar - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24005,Folha Mensal - Setembro,ZAIRA ANGELINA ROGADO,AUXILIAR DE CRECHE,09/12/2013,REGIME PROPRIO,"323,08","2.442,78",NaN,"2.765,86",1.0,NaN,NaN,Efetivo,NaN,40.0,set2025.csv
24006,Folha Mensal - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,"1.947,27","3.576,00",NaN,"5.523,27",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv
24007,Folha Complementar - Setembro,ZILDA MESSINA,AUXILIAR DE ENFERMAGEM,02/05/1994,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,NaN,NaN,Efetivo,NaN,30.0,set2025.csv


In [27]:
# Extrair tipo de pagamento e mês diretamente da coluna referencia
df_prepared[["tipo_pagamento_raw", "mes_referencia"]] = df_prepared["referencia"].str.split(" - ", expand=True)

# Padronizar tipo de pagamento
df_prepared["tipo_pagamento"] = df_prepared["tipo_pagamento_raw"].replace({
    "Folha Mensal": "folha_mensal",
    "Folha Complementar": "vale_alimentacao"
})

# remover coluna temporária
df_prepared = df_prepared.drop(columns=["tipo_pagamento_raw"])

# conferir resultados
df_prepared[["referencia", "tipo_pagamento", "mes_referencia"]].head()

,referencia,tipo_pagamento,mes_referencia
0,Folha Complementar - Abril,vale_alimentacao,Abril
1,Folha Mensal - Abril,folha_mensal,Abril
2,Folha Complementar - Abril,vale_alimentacao,Abril
3,Folha Mensal - Abril,folha_mensal,Abril
4,Folha Complementar - Abril,vale_alimentacao,Abril


In [28]:
df_prepared["tipo_pagamento"].unique()

array(['vale_alimentacao', 'folha_mensal', 'Adiantamento 13º Salário',
       'Folha Complementar c/ Encargos', 'Rescisão',
       'Fechamento 13º Salário'], dtype=object)

In [29]:
def normalizar_tipo_pagamento(valor):
    if pd.isna(valor):
        return valor
    
    mapeamento = {
        "vale_alimentacao": "vale_alimentacao",
        "folha_mensal": "folha_mensal",
        "Adiantamento 13º Salário": "adiantamento_13_salario",
        "Folha Complementar c/ Encargos": "folha_complementar_com_encargos",
        "Rescisão": "rescisao",
        "Fechamento 13º Salário": "fechamento_13_salario"
    }

    return mapeamento.get(valor, valor)

In [30]:
df_prepared["tipo_pagamento"].apply(normalizar_tipo_pagamento)

0        vale_alimentacao
1            folha_mensal
2        vale_alimentacao
3            folha_mensal
4        vale_alimentacao
               ...       
24003        folha_mensal
24004    vale_alimentacao
24005        folha_mensal
24006        folha_mensal
24007    vale_alimentacao
Name: tipo_pagamento, Length: 23997, dtype: object

In [31]:
df_prepared["tipo_pagamento"].unique()

array(['vale_alimentacao', 'folha_mensal', 'Adiantamento 13º Salário',
       'Folha Complementar c/ Encargos', 'Rescisão',
       'Fechamento 13º Salário'], dtype=object)

In [32]:
df_prepared["tipo_pagamento_normalizado"] = df_prepared["tipo_pagamento"].apply(normalizar_tipo_pagamento)

In [33]:
df_prepared["tipo_pagamento_normalizado"].value_counts()

tipo_pagamento_normalizado
folha_mensal                       11005
vale_alimentacao                   10937
adiantamento_13_salario              926
fechamento_13_salario                872
rescisao                             155
folha_complementar_com_encargos      102
Name: count, dtype: int64

## Inferência manual do sexo dos servidores — Processo

### Extração dos nomes
- A partir do dataset preparado, foram extraídos os nomes dos servidores.
- Os nomes foram deduplicados para garantir que cada servidor fosse analisado apenas uma vez.

### Normalização dos nomes
- Os nomes foram padronizados para facilitar comparações posteriores.
- A normalização consistiu em:
  - Conversão para letras maiúsculas
  - Remoção de espaços em branco no início e no fim do texto

### Criação de DataFrame auxiliar
- Foi criado um DataFrame exclusivo contendo:
  - Nome do servidor
  - Campo vazio para inferência manual do sexo
- Esse DataFrame passou a ser a fonte oficial da variável inferida.

### Exportação para CSV
- O DataFrame auxiliar foi exportado para um arquivo CSV.
- O objetivo foi permitir uma inferência manual mais eficiente em ambiente visual (ex.: Excel ou LibreOffice).

### Inferência manual do sexo
- Cada nome foi analisado individualmente.
- O sexo foi inferido com base no primeiro nome do servidor.
- Foi adotada a seguinte convenção:
  - `M` para masculino
  - `F` para feminino

### Reimportação do arquivo tratado
- Após a inferência manual, o arquivo CSV foi reimportado para o ambiente de análise.
- A normalização dos nomes foi reaplicada para garantir consistência.

### Integração com o dataset principal
- A variável `sexo_inferido` foi incorporada ao dataset principal por meio de *merge*.
- A junção foi realizada utilizando o nome normalizado como chave.

### Validação final
- Foi realizada a verificação de valores nulos na variável `sexo_inferido`.
- Caso identificados, os registros foram corrigidos diretamente no CSV e o processo de importação e *merge* foi repetido.


In [34]:
nomes_unicos = (
    df_prepared["nome"]
    .dropna()
    .drop_duplicates()
    .sort_values()
    .reset_index(drop=True)
)

nomes_unicos

0      ACACIO DONIZETTI DE OLIVEIRA
1                 ADA ESTER ARCHILA
2           ADAO RODRIGUES DE JESUS
3                 ADELINI PELINGRIN
4                ADEMILTON MONTEIRO
                   ...             
975            WILSON RANOEL VERGNA
976               YARA MARIA ISABEL
977          YASMIN MICKAELA TAMINI
978           ZAIRA ANGELINA ROGADO
979                   ZILDA MESSINA
Name: nome, Length: 980, dtype: object

In [35]:
df_sexo_servidores = pd.DataFrame({
    "nome_servidor": nomes_unicos,
    "sexo_inferido": pd.NA
})

df_sexo_servidores

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,<NA>
1,ADA ESTER ARCHILA,<NA>
2,ADAO RODRIGUES DE JESUS,<NA>
3,ADELINI PELINGRIN,<NA>
4,ADEMILTON MONTEIRO,<NA>
...,...,...
975,WILSON RANOEL VERGNA,<NA>
976,YARA MARIA ISABEL,<NA>
977,YASMIN MICKAELA TAMINI,<NA>
978,ZAIRA ANGELINA ROGADO,<NA>


In [36]:
path_output = Path("../data/interim/sexo_servidores.csv")

df_sexo_servidores.to_csv(
    path_output,
    index=False,
    encoding="utf-8"
)

In [37]:
df_sexo_serv = pd.read_csv("../data/interim/inf_sexo_servidor.csv", sep=";")

df_sexo_serv.head()

,nome_servidor,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,M
1,ADA ESTER ARCHILA,F
2,ADAO RODRIGUES DE JESUS,M
3,ADELINI PELINGRIN,F
4,ADEMILTON MONTEIRO,M


In [38]:
def normalizar_nome(nome):
    return (
        nome
        .str.upper()
        .str.strip()
    )

df_prepared["nome_servidor_norm"] = normalizar_nome(df_prepared["nome"])
df_sexo_serv["nome_servidor_norm"] = normalizar_nome(df_sexo_serv["nome_servidor"])

In [39]:
df_prepared = df_prepared.merge(
    df_sexo_serv[["nome_servidor_norm", "sexo_inferido"]],
    on="nome_servidor_norm",
    how="left"
)

In [40]:
df_prepared["sexo_inferido"].value_counts(dropna=False)

sexo_inferido
F    14784
M     9213
Name: count, dtype: int64

In [41]:
df_prepared[["nome", "sexo_inferido"]].head()

,nome,sexo_inferido
0,ACACIO DONIZETTI DE OLIVEIRA,M
1,ACACIO DONIZETTI DE OLIVEIRA,M
2,ADA ESTER ARCHILA,F
3,ADA ESTER ARCHILA,F
4,ADAO RODRIGUES DE JESUS,M


## Anonimização dos dados dos servidores

A partir deste ponto, os nomes dos servidores deixam de ser utilizados diretamente no processo de análise.

Embora os dados analisados sejam públicos, a anonimização é adotada como **boa prática em Ciência de Dados**, especialmente quando o objetivo é:

- Focar em padrões, distribuições e comportamentos agregados
- Evitar exposição desnecessária de informações pessoais
- Tornar o projeto reutilizável em outros contextos e datasets sensíveis

### Estratégia adotada

A anonimização foi realizada por meio da **geração de um identificador único irreversível** para cada servidor, utilizando uma função de hash criptográfico.

Características da abordagem:

- O mesmo nome sempre gera o mesmo identificador
- O processo é determinístico
- Não é possível recuperar o nome original a partir do identificador
- O identificador pode ser usado com segurança como chave em análises futuras

### Algoritmo utilizado

Foi utilizado o algoritmo **SHA-256**, disponível na biblioteca padrão do Python (`hashlib`), por ser amplamente adotado e seguro para este tipo de aplicação.

### Impacto na análise

Após a criação do identificador anonimizado:

- O nome do servidor deixa de ser necessário para análises estatísticas
- As análises passam a utilizar exclusivamente o identificador (`id_servidor`)
- Variáveis como sexo inferido, remuneração e período permanecem preservadas

Essa abordagem garante um equilíbrio entre **qualidade analítica**, **reprodutibilidade** e **responsabilidade no uso dos dados**.


In [42]:
def gerar_id_servidor(nome):
    if pd.isna(nome):
        return None
    return hashlib.sha256(nome.encode("utf-8")).hexdigest()

df_prepared["id_servidor"] = df_prepared["nome_servidor_norm"].apply(gerar_id_servidor)

In [43]:
df_prepared[["nome_servidor_norm", "id_servidor"]].head()

,nome_servidor_norm,id_servidor
0,ACACIO DONIZETTI DE OLIVEIRA,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...
1,ACACIO DONIZETTI DE OLIVEIRA,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...
2,ADA ESTER ARCHILA,348b58a2d1473088be45887aaa8b05de2944865139083c...
3,ADA ESTER ARCHILA,348b58a2d1473088be45887aaa8b05de2944865139083c...
4,ADAO RODRIGUES DE JESUS,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...


## Tratando a coluna cargo

In [44]:
df_prepared["cargo"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: cargo
Non-Null Count  Dtype 
--------------  ----- 
23997 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


Nesta coluna foi identificado 23997 linhas sem valores nulos

### Quantidade de cargos distintos

In [45]:
df_prepared["cargo"].nunique()

127

In [46]:
sorted(df_prepared["cargo"].unique())

['AGENTE ADMINISTRATIVO',
 'AGENTE COMUNITARIO DE SAUDE.',
 'AGENTE DE COMBATE AS ENDEMIAS',
 'AGENTE DE DESENVOLVIMENTO INFANTIL',
 'AGENTE DE SERVIÇOS DE SAÚDE',
 'AGENTE DE VIGILANCIA SANITARIA',
 'AJUDANTE DE ENCANADOR',
 'AJUDANTE DE PEDREIRO',
 'AJUDANTE DE SERVICOS DIVERSOS',
 'ASSESSOR DE GABINETE DE DIRETOR DE DEPARTAMENTO.c',
 'ASSESSOR DE GABINETE.c',
 'ASSESSOR DE IMPLEMENTAÇÃO DE POLÍTICAS PÚBLICAS.c',
 'ASSESSOR DE PLANEJAMENTO.c',
 'ASSISTENTE SOCIAL',
 'ATENDENTE SOCIAL',
 'AUXILIAR DE CRECHE',
 'AUXILIAR DE CUIDADOR DE CRIANCA - ABRIGO INSTITUCIONAL',
 'AUXILIAR DE DENTISTA PSF',
 'AUXILIAR DE ENFERMAGEM',
 'AUXILIAR DE FARMACIA',
 'AUXILIAR DE MANUTENCAO',
 'AUXILIAR DE MEDICOS DENTISTA',
 'AUXILIAR DE SERVICOS EXTERNOS',
 'BIBLIOTECARIA',
 'BORRACHEIRO',
 'CARPINTEIRO',
 'CHEFE DE GABINETE.c',
 'CIRURGIAO DENTISTA PSF',
 'CONSELHEIRO TUTELAR.',
 'CONTADOR',
 'CONTROLADOR INTERNO',
 'COVEIRO',
 'DENTISTA',
 'DIRETOR DO DEP. TURISMO, DESEN. ECO., CULTURA E ESPORTES.c',

- A listagem completa dos cargos foi inspecionada visualmente para verificação de consistência e padronização.
- Cargo é considerado comissionado quando o nome do cargo termina com “.c”.

## Criando categoria por cargo

- Normalizando a coluna cargo

In [47]:
def normalizar_texto(texto):
    if pd.isna(texto):
        return texto

    texto = unicodedata.normalize("NFKD", texto)
    texto = texto.encode("ASCII", "ignore").decode("ASCII")

    texto = texto.upper()

    texto = texto.strip()
    texto = re.sub(r"\s+", " ", texto)


    return texto

df_prepared["cargo_norm"] = df_prepared["cargo"].apply(normalizar_texto)

In [48]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,data_prevista_termino_contrato,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,NaN,40.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,NaN,30.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,NaN,30.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,NaN,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO


- Mapeando os cargos

In [49]:
mapa_categoria_cargo = {
    # ADMINISTRATIVO / GESTAO
    "AGENTE ADMINISTRATIVO": "administrativo",
    "OFICIAL ADMINISTRATIVO": "administrativo",
    "CONTADOR": "administrativo",
    "CONTROLADOR INTERNO": "administrativo",
    "GESTOR DE PLANEJAMENTO": "administrativo",
    "FISCAL": "administrativo",

    # SAUDE
    "AGENTE COMUNITARIO DE SAUDE.": "saude",
    "AGENTE DE COMBATE AS ENDEMIAS": "saude",
    "AGENTE DE SERVICOS DE SAUDE": "saude",
    "AGENTE DE VIGILANCIA SANITARIA": "saude",
    "AUXILIAR DE DENTISTA PSF": "saude",
    "AUXILIAR DE ENFERMAGEM": "saude",
    "AUXILIAR DE FARMACIA": "saude",
    "AUXILIAR DE MEDICOS DENTISTA": "saude",
    "CIRURGIAO DENTISTA PSF": "saude",
    "DENTISTA": "saude",
    "ENFERMEIRO": "saude",
    "FARMACEUTICO": "saude",
    "FISIOTERAPEUTA": "saude",
    "FONOAUDIOLOGA": "saude",
    "MEDICO": "saude",
    "MEDICO ANESTESISTA": "saude",
    "MEDICO CARDIOLOGISTA": "saude",
    "MEDICO CIRURGIAO": "saude",
    "MEDICO CLINICO GERAL": "saude",
    "MEDICO DO TRABALHO": "saude",
    "MEDICO ENDOCRINOLOGISTA": "saude",
    "MEDICO GINECOLOGISTA/OBSTETRA": "saude",
    "MEDICO NEUROLOGISTA ADULTO": "saude",
    "MEDICO ORTOPEDISTA / TRAUMATOLOGISTA": "saude",
    "MEDICO PEDIATRA": "saude",
    "MEDICO PRONTO ATENDIMENTO": "saude",
    "MEDICO PSF": "saude",
    "MEDICO PSIQUIATRA ADULTO": "saude",
    "MEDICO UROLOGISTA": "saude",
    "NUTRICIONISTA": "saude",
    "TECNICO EM ENFERMAGEM": "saude",
    "TECNICO EM NUTRICAO": "saude",
    "VETERINARIO": "saude",

    # EDUCACAO
    "AGENTE DE DESENVOLVIMENTO INFANTIL": "educacao",
    "AUXILIAR DE CRECHE": "educacao",
    "INSPETOR DE ALUNOS": "educacao",
    "MERENDEIRA": "educacao",
    "MONITOR DE EDUCACAO FISICA": "educacao",
    "MESTRE DE MUSICA": "educacao",
    "PROFESSOR": "educacao",
    "PROFESSOR DE EDUCACAO FISICA": "educacao",
    "PROFESSOR DE EDUCACAO BASICA I - PEB I": "educacao",
    "PROFESSOR DE EDUCACAO BASICA I - PEB I - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - ARTE - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - CIENCIAS - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - EDUCACAO FISICA PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - GEOGRAFIA - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - INFORMATICA": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - INGLES - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - MATEMATICA - PD": "educacao",
    "PROFESSOR DE EDUCACAO BASICA II - PORTUGUES - PD": "educacao",
    "PROFESSOR DE EDUCACAO INFANTIL - PEI - PD": "educacao",
    "PROFESSOR DE EDUCACAO INFANTIL -PEI": "educacao",
    "PROFESSOR DE ENSINO ESPECIAL": "educacao",
    "PROFESSOR DE SALA DE APOIO (PSA) EDUCACAO ESPECIAL - PD": "educacao",
    "PROFESSOR DE SALA DE APOIO (PSA-EDUC.ESPECIAL)": "educacao",
    "PROFESSOR EDUC.INFANTIL(CRECHE)-PEI-C": "educacao",
    "PROFESSOR EDUCACAO BASICA II - ARTE": "educacao",
    "PROFESSOR EDUCACAO BASICA II - EDUCACAO FISICA": "educacao",
    "PROFESSOR EDUCACAO BASICA II - GEOGRAFIA": "educacao",
    "PROFESSOR EDUCACAO BASICA II - LINGUA PORTUGUESA": "educacao",
    "PROFESSOR ENS FUND-CICLO I - II": "educacao",
    "PROFESSOR ENS FUND-CICLO III - IV": "educacao",
    "PROFESSOR SALA DE APOIO (PSA) EDUCACAO ESPECIAL": "educacao",

    # ASSISTENCIA SOCIAL
    "ASSISTENTE SOCIAL": "assistencia_social",
    "ATENDENTE SOCIAL": "assistencia_social",
    "ORIENTADOR SOCIAL": "assistencia_social",
    "CONSELHEIRO TUTELAR.": "assistencia_social",
    "AUXILIAR DE CUIDADOR DE CRIANCA - ABRIGO INSTITUCIONAL": "assistencia_social",
    "SUPERVISOR DE VISITAS": "assistencia_social",
    "PSICOLOGO": "assistencia_social",
    "PSICOLOGO INFANTIL": "assistencia_social",

    # OPERACIONAL / SERVICOS GERAIS
    "AJUDANTE DE ENCANADOR": "operacional",
    "AJUDANTE DE PEDREIRO": "operacional",
    "AJUDANTE DE SERVICOS DIVERSOS": "operacional",
    "AUXILIAR DE MANUTENCAO": "operacional",
    "AUXILIAR DE SERVICOS EXTERNOS": "operacional",
    "BORRACHEIRO": "operacional",
    "CARPINTEIRO": "operacional",
    "COVEIRO": "operacional",
    "ELETRICISTA": "operacional",
    "ENCANADOR": "operacional",
    "JARDINEIRO": "operacional",
    "LEITURISTA": "operacional",
    "LIXEIRO": "operacional",
    "MECANICO II": "operacional",
    "MOTORISTA": "operacional",
    "OPERADOR DA EBA": "operacional",
    "OPERADOR DE MAQUINA II": "operacional",
    "OPERADOR DE VACA MECANICA": "operacional",
    "PADEIRO": "operacional",
    "PEDREIRO": "operacional",
    "SERVENTE": "operacional",
    "VIGIA": "operacional",

    # TECNICO
    "TECNICO EM INFORMATICA": "tecnico",
    "TECNICO EM QUIMICA INDUSTRIAL (ETE)": "tecnico",
    "TECNICO SEGURANCA DO TRABALHO": "tecnico",

    # CULTURA / TURISMO
    "BIBLIOTECARIA": "cultura",
    "TURISMOLOGO": "cultura",

    # JURIDICO
    "PROCURADOR JURIDICO": "juridico",

    # POLITICO / ALTA GESTAO
    "PREFEITO": "politico",
    "VICE PREFEITO": "politico",

    # COMISSIONADOS / CARGOS DE CONFIANCA
    "ASSESSOR DE GABINETE DE DIRETOR DE DEPARTAMENTO.C": "comissionado",
    "ASSESSOR DE GABINETE.C": "comissionado",
    "ASSESSOR DE IMPLEMENTACAO DE POLITICAS PUBLICAS.C": "comissionado",
    "ASSESSOR DE PLANEJAMENTO.C": "comissionado",
    "CHEFE DE GABINETE.C": "comissionado",
    "GESTOR ADJUNTO DE ENSINO FUNDAMENTAL.C": "comissionado",
    "PROCURADOR GERAL DO MUNICIPIO.C": "comissionado",
    "DIRETOR DO DEP. TURISMO, DESEN. ECO., CULTURA E ESPORTES.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE ADMINISTRACAO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE AGRICULTURA E MEIO AMBIENTE.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE ASSISTENCIA SOCIAL.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE DESENVOLVIMENTO ECONOMICO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE DESENVOLVIMENTO URBANO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE EDUCACAO.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE FINANCAS.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE GESTAO DE PESSOAS.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE OBRAS E ENGENHARIA.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE SAUDE.C": "comissionado",
    "DIRETOR DO DEPARTAMENTO DE SERVICOS MUNICIPAIS.C": "comissionado",
}


In [50]:
df_prepared["categoria_cargo"] = df_prepared["cargo_norm"].map(mapa_categoria_cargo)

In [51]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,carga_horaria_semanal,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,40.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,30.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,30.0,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,40.0,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional


- Validando se algum cargo ficou sem catgoria

In [52]:
# Validando se algum cargo ficou sem categoria

df_prepared["categoria_cargo"].isna().sum()

np.int64(0)

In [53]:
(
    df_prepared
    .loc[df_prepared["categoria_cargo"].isna(), "cargo_norm"]
    .drop_duplicates()
    .sort_values()
)


Series([], Name: cargo_norm, dtype: object)

In [54]:
(
    df_prepared["categoria_cargo"]
    .isna()
    .mean()
    * 100
)


np.float64(0.0)

In [55]:
df_prepared["cargo_categorizado"] = df_prepared["categoria_cargo"].notna()

In [56]:
df_prepared["cargo_categorizado"].value_counts()

cargo_categorizado
True    23997
Name: count, dtype: int64

In [57]:
(
    df_prepared
    .loc[df_prepared["categoria_cargo"].isna(), "cargo_norm"]
    .drop_duplicates()
    .sort_values()
)


Series([], Name: cargo_norm, dtype: object)

In [58]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,08/04/1991,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,04/12/2023,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,08/04/1991,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


- Conforme visto acima, todos os cargos foram mapeados

## Tratando coluna data_admissao

- Verificando nulos

In [59]:
df_prepared["data_admissao"].isna().sum()

np.int64(0)

In [60]:
df_prepared["data_admissao"].describe()

count          23997
unique           440
top       03/02/2025
freq            1066
Name: data_admissao, dtype: object

- Transformando o tipo da coluna para date

In [61]:
df_prepared["data_admissao"] = pd.to_datetime(
    df_prepared["data_admissao"],
    errors="coerce",
    dayfirst=True
)

In [62]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,arquivo_origem,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,abr2025.csv,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,abr2025.csv,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True


In [63]:
type(df_prepared['data_admissao'].dropna().iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

## Criação da flag de identificação de agentes políticos

Durante a análise dos dados da folha de pagamento, observou-se que os cargos de **Prefeito** e **Vice-Prefeito** apresentam características administrativas e remuneratórias distintas dos demais servidores públicos, por se tratarem de **agentes políticos**. Além disso, esses cargos podem apresentar registros com datas de desligamento que não representam a saída efetiva do cargo, mas sim o encerramento administrativo de vínculos no sistema de folha.

Para lidar adequadamente com essa particularidade, foi criada a variável auxiliar `flag_agente_politico`, com o objetivo de identificar registros referentes a cargos eletivos, sem removê-los da base de dados.

A flag assume os seguintes valores:
- `True`: registros cujo cargo corresponde a Prefeito ou Vice-Prefeito
- `False`: demais servidores públicos

Essa abordagem permite:
- Preservar a integridade do dataset original
- Aplicar regras específicas para agentes políticos de forma controlada
- Excluir esses cargos apenas em análises exploratórias e visualizações, quando necessário, evitando distorções estatísticas
- Diferenciar vínculos administrativos de mudanças reais de ocupação do cargo

A identificação foi realizada a partir da coluna de cargo, utilizando correspondência textual para o termo “PrefEITO”, de modo a abranger variações como “Prefeito” e “Vice-Prefeito”.


In [64]:
df_prepared["flag_agente_politico"] = df_prepared["cargo"].str.contains(
    r'\bPREFEITO\b',
    case=False,
    na=False
)

In [65]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,"1.584,02","3.603,75",NaN,"5.187,77",1.0,...,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,"1.330,84","5.432,23",NaN,"6.763,07",1.0,...,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,NaN,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False


In [66]:
df_prefeito_vice = df_prepared[df_prepared['flag_agente_politico'] == True]

df_prefeito_vice[['nome', 'cargo', 'data_admissao', 'data_desligamento']].drop_duplicates()

,nome,cargo,data_admissao,data_desligamento
285,ARIANA APARECIDA PUPO,VICE PREFEITO,2025-01-01,NaN
1256,MARCELO SIMAO,PREFEITO,2025-01-01,NaN
9685,MARCELO SIMAO,PREFEITO,2021-01-01,01/01/2025
9706,MARCIO RENATO CANDIDO DOS REIS,VICE PREFEITO,2021-01-01,01/01/2025


## Tratando coluna tipo_regime

In [67]:
df_prepared["tipo_regime"].describe

<bound method NDFrame.describe of 0        REGIME PROPRIO
1        REGIME PROPRIO
2        REGIME PROPRIO
3        REGIME PROPRIO
4        REGIME PROPRIO
              ...      
23992    REGIME PROPRIO
23993    REGIME PROPRIO
23994    REGIME PROPRIO
23995    REGIME PROPRIO
23996    REGIME PROPRIO
Name: tipo_regime, Length: 23997, dtype: object>

In [68]:
sorted(df_prepared["tipo_regime"].unique())

['REGIME GERAL  ', 'REGIME PROPRIO']

In [69]:
df_prepared["tipo_regime"] = df_prepared["tipo_regime"].apply(normalizar_texto)

In [70]:
df_prepared["tipo_regime"].unique()

array(['REGIME PROPRIO', 'REGIME GERAL'], dtype=object)

No contexto de servidores públicos municipais/estaduais:

Regime Geral (geralmente “RGPS” – Regime Geral de Previdência Social):
Normalmente se refere a servidores efetivos, que ingressaram via concurso público.
Contribuem para o sistema de previdência geral (INSS).
São considerados ocupantes de cargos estáveis, com direitos típicos de servidores concursados.


Regime Próprio (geralmente “RPPS” – Regime Próprio de Previdência Social):
Pode incluir servidores comissionados (cargos de confiança) ou efetivos, dependendo do município/estado.
Para cargos comissionados, geralmente não há estabilidade e podem ser exonerados a qualquer momento.
Contribuem para o regime próprio de previdência do ente federativo (municipal ou estadual).

Sendo assim, será mantido o formato da coluna e não passará por nenhuma transformação

### Tratamento da coluna `desconto`

A coluna `desconto` representa os valores descontados do pagamento do servidor.

Observações iniciais:
- Quando o pagamento é referente ao **vale alimentação**, não há desconto, portanto a coluna apresenta `NaN`.
- Quando o pagamento é referente ao **salário (folha mensal)**, o valor do desconto vem no formato brasileiro, por exemplo: `1.500,00`.

#### Objetivos do tratamento:
1. Zerar a coluna `desconto` para os pagamentos do tipo **vale alimentação**.
2. Converter os valores de salário para o formato **float**, pronto para análises financeiras.
3. Garantir que não haja valores nulos (`NaN`) na coluna após o tratamento.

#### Passo a passo:

1. **Identificar o tipo de pagamento**
   - A coluna `tipo_pagamento` indica se o registro é `folha_mensal` ou `vale_alimentacao`.

2. **Função para conversão de string pt-br para float**

In [71]:
def str_para_float(valor):
    if pd.isna(valor):
        return 0.0

    return float(str(valor).replace(".", "").replace(",", "."))

df_prepared["descontos"] = np.where(
    df_prepared["tipo_pagamento"] == "vale_alimentacao", 0.0,
    df_prepared["descontos"].apply(str_para_float)
)

In [72]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,"3.603,75",NaN,"5.187,77",1.0,...,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,"5.432,23",NaN,"6.763,07",1.0,...,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,"1.000,00",NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False


In [73]:
df_prepared["descontos"].describe

<bound method NDFrame.describe of 0           0.00
1        1584.02
2           0.00
3        1330.84
4           0.00
          ...   
23992     535.56
23993       0.00
23994     323.08
23995    1947.27
23996       0.00
Name: descontos, Length: 23997, dtype: float64>

In [74]:
df_prepared["descontos"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: descontos
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


### Tratamento da coluna `liquido`

A coluna `liquido` representa o valor líquido pago ao servidor em cada pagamento.

Observações iniciais:
- Os valores estão no **formato brasileiro**, por exemplo `1.500,00`.
- Para análises financeiras, precisamos do formato **float**, por exemplo `1500.00`.
- Não há necessidade de diferenciar por tipo de pagamento; todos os registros devem ser convertidos.

#### Objetivos do tratamento:
1. Converter os valores para **float**.
2. Garantir que não haja valores `NaN`.
3. Padronizar para análises financeiras e cálculos futuros.

In [75]:
df_prepared["liquido"].describe

<bound method NDFrame.describe of 0        1.000,00
1        3.603,75
2        1.000,00
3        5.432,23
4        1.000,00
           ...   
23992    2.920,63
23993    1.000,00
23994    2.442,78
23995    3.576,00
23996    1.000,00
Name: liquido, Length: 23997, dtype: object>

In [76]:
df_prepared["liquido"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: liquido
Non-Null Count  Dtype 
--------------  ----- 
23997 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [77]:
def str_para_float(valor):
    if pd.isna(valor):
        return 0.0
    # Substitui ponto como separador de milhar e vírgula como decimal
    return float(str(valor).replace('.', '').replace(',', '.'))

df_prepared["liquido"] = df_prepared["liquido"].apply(str_para_float)

In [78]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,mes_referencia,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,Abril,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,Abril,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,Abril,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False


## Tratamento da coluna `data_desligamento`

A coluna `data_desligamento` indica a data em que o servidor foi desligado do cargo. Após análise, observou-se que a maioria dos registros apresenta valor nulo (`NaN`), pois servidores públicos geralmente permanecem ativos por longos períodos, sendo que desligamentos ocorrem apenas em casos de cargos comissionados ou situações fora do padrão.

**Processo de tratamento aplicado:**

1. Foi criada uma nova coluna `data_desligamento_formatada` para manter a informação original intacta.
2. Para registros com valor nulo (`NaN`), foi definido o símbolo `"-"` indicando que o servidor está ativo.
3. Para registros com data, o valor foi convertido para o tipo `date`, garantindo consistência e formato padrão (`YYYY-MM-DD`).
4. Essa abordagem permite análises futuras, como identificação de servidores desligados, cálculo de tempo de serviço, entre outros, sem perder a informação original.

Exemplo de resultado:

| data_desligamento | data_desligamento_formatada |
|------------------|----------------------------|
| NaN              | -                          |
| 15/08/2020       | 2020-08-15                 |


In [79]:
def formatar_desligamento(data):
    if pd.isna(data):
        return pd.NaT
    else:
        return pd.to_datetime(data, dayfirst=True).date()

df_prepared["data_desligamento_formatada"] = df_prepared["data_desligamento"].apply(formatar_desligamento)

In [80]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_formatada
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,folha_mensal,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,folha_mensal,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False,NaT


In [81]:
df_prepared["data_desligamento_formatada"].nunique()

73

In [82]:
df_prepared["data_desligamento_formatada"].value_counts()

data_desligamento_formatada
2025-12-12    54
2025-06-02     7
2025-01-01     4
2025-07-01     4
2025-05-05     4
              ..
2025-10-02     1
2025-09-09     1
2025-09-25     1
2025-09-26     1
2025-09-01     1
Name: count, Length: 73, dtype: int64

In [83]:
df_prepared["status_servidor"] = df_prepared["data_desligamento"].apply(
    lambda x: "ATIVO" if pd.isna(x) else "DESLIGADO"
)

In [84]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_formatada,status_servidor
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,"5.187,77",1.0,...,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,"6.763,07",1.0,...,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,"1.000,00",1.0,...,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False,NaT,ATIVO


- Servidores desligados

Após análise da coluna `data_desligamento_formatada`, foi identificado que **149 servidores** possuem registro de desligamento no período analisado.  
Para garantir consistência, a contagem considera **apenas servidores únicos**, evitando duplicidade em casos onde um servidor possui múltiplos registros de pagamento.

### Tratamento da coluna `proventos`

A coluna `proventos` representa o salário bruto do servidor e inicialmente estava no formato `object` com notação brasileira de números (ex.: `1.500,00`).

O tratamento realizado consistiu em:

1. Remover o ponto que separa os milhares.
2. Substituir a vírgula pelo ponto para padronizar o decimal.
3. Converter a coluna para tipo `float`.

Após esse tratamento, a coluna está pronta para análises quantitativas, cálculos de média, soma, etc.

In [85]:
df_prepared["proventos"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: proventos
Non-Null Count  Dtype 
--------------  ----- 
23919 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [86]:
df_prepared["proventos"] = (
    df_prepared["proventos"]
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

In [87]:
df_prepared.head()

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento_normalizado,nome_servidor_norm,sexo_inferido,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_formatada,status_servidor
0,Folha Complementar - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,1000.00,1.0,...,vale_alimentacao,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
1,Folha Mensal - Abril,ACACIO DONIZETTI DE OLIVEIRA,MECANICO II,1991-04-08,REGIME PROPRIO,1584.02,3603.75,NaN,5187.77,1.0,...,folha_mensal,ACACIO DONIZETTI DE OLIVEIRA,M,5a69806a2b26328cb82b24260282ba8037ddf3a430cff8...,MECANICO II,operacional,True,False,NaT,ATIVO
2,Folha Complementar - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,0.00,1000.00,NaN,1000.00,1.0,...,vale_alimentacao,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
3,Folha Mensal - Abril,ADA ESTER ARCHILA,TECNICO EM ENFERMAGEM,2023-12-04,REGIME PROPRIO,1330.84,5432.23,NaN,6763.07,1.0,...,folha_mensal,ADA ESTER ARCHILA,F,348b58a2d1473088be45887aaa8b05de2944865139083c...,TECNICO EM ENFERMAGEM,saude,True,False,NaT,ATIVO
4,Folha Complementar - Abril,ADAO RODRIGUES DE JESUS,JARDINEIRO,1991-04-08,REGIME PROPRIO,0.00,1000.00,NaN,1000.00,1.0,...,vale_alimentacao,ADAO RODRIGUES DE JESUS,M,28b810f6dcab5e62dcaddb7d93fa4384c7858119885ee4...,JARDINEIRO,operacional,True,False,NaT,ATIVO


## Coluna `contrato`

A coluna `contrato` apresenta apenas dois valores distintos: 1.0 e 2.0.  
Em bases de servidores públicos, é comum que números assim sejam códigos internos utilizados para diferenciar tipos de vínculo:

- **1** → Contrato efetivo ou concursado (regime estatutário ou CLT de longo prazo)  
- **2** → Contrato temporário ou vínculo especial (temporário, emergencial ou comissionado)  

In [88]:
df_prepared["contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: contrato
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


In [89]:
df_prepared["contrato"] = df_prepared["contrato"].astype("Int64")

In [90]:
df_prepared["contrato"].value_counts()

contrato
1    22714
2     1283
Name: count, dtype: Int64

**Decisão adotada:**  
A coluna `contrato` não será utilizada para análises.

## Coluna `atividade`

A coluna `atividade` apresenta poucos registros preenchidos.  
Quando preenchida, contém apenas números, o que indica que provavelmente se trata de **códigos internos** da prefeitura para classificar funções ou atividades específicas.  

**Observações:**  
- Não há clareza sobre o que cada código representa.  
- A coluna não parece ser relevante para as análises que estamos planejando.  

In [91]:
df_prepared["atividade"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: atividade
Non-Null Count  Dtype  
--------------  -----  
24 non-null     float64
dtypes: float64(1)
memory usage: 187.6 KB


In [92]:
df_prepared["atividade"] = df_prepared["atividade"].astype("Int64")

In [93]:
df_prepared["atividade"].value_counts()

atividade
10    24
Name: count, dtype: Int64

**Decisão adotada:**  
A coluna `atividade` não irá ser usada nas análises.  
Se futuramente for identificado o significado dos códigos, poderemos mapear ou categorizar esta coluna.

## Coluna `nome_atividade`

A coluna `nome_atividade` apresenta poucos registros preenchidos.  
Quando preenchida, contém informações que **já estão presentes na coluna `cargo`**, ou seja, não agrega valor adicional para nossas análises.  

In [94]:
df_prepared["nome_atividade"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: nome_atividade
Non-Null Count  Dtype 
--------------  ----- 
24 non-null     object
dtypes: object(1)
memory usage: 187.6+ KB


**Decisão adotada:**  
Assim como a coluna `atividade`, **não será realizado nenhum tratamento** na coluna `nome_atividade` neste momento.  
Caso futuramente seja identificado algum uso específico para esses dados, poderemos revisitar a coluna.

## Coluna `tipo_contrato`

A coluna `tipo_contrato` apresenta os seguintes valores:

- Efetivo  
- Contrato Temporario  
- Função de Confiança  
- Efetivo em Comissão  

In [95]:
df_prepared["tipo_contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: tipo_contrato
Non-Null Count  Dtype 
--------------  ----- 
23984 non-null  object
dtypes: object(1)
memory usage: 187.6+ KB


In [96]:
df_prepared["tipo_contrato"].unique()

array(['Efetivo', 'Função de Confiança', 'Contrato Temporario',
       'Efetivo em Comissão', nan, 'Contrato/Temporário'], dtype=object)

**Observações:**

- Existem poucos valores nulos.
- A maioria dos registros estão consistentes e padronizados.  

**Decisão adotada:**  
No momento, **nenhum tratamento será realizado** na coluna, pois os dados já estão prontos para análise.
Caso futuramente seja identificado algum uso específico para esses dados, poderemos revisitar a coluna.

## Coluna `data_prevista_termino_contrato`

In [97]:
df_prepared["data_prevista_termino_contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: data_prevista_termino_contrato
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 187.6 KB


In [98]:
df_prepared["data_prevista_termino_contrato"] = pd.to_datetime(
    df_prepared["data_prevista_termino_contrato"],
    errors="coerce",
    dayfirst=True
)

In [99]:
df_prepared["data_prevista_termino_contrato"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: data_prevista_termino_contrato
Non-Null Count  Dtype         
--------------  -----         
0 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 187.6 KB


Observações:

- Todos os valores da coluna estão nulos.  
- Não existe nenhuma informação preenchida em nenhuma linha.

**Decisão adotada**:

- A coluna será mantida no dataset, mas **não será utilizada nas análises**, pois não contém dados relevantes.  
- Caso seja necessário futuramente, poderá ser descartada ou ignorada sem impacto.

## Coluna `carga_horaria_semanal`

Observações:

- A coluna informa a carga horária semanal de cada servidor.  
- Atualmente está no formato `float`, mas os valores representam horas inteiras.

Tratamento realizado:

- A coluna foi convertida de `float` para `int` para facilitar futuras análises.

In [100]:
df_prepared["carga_horaria_semanal"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: carga_horaria_semanal
Non-Null Count  Dtype  
--------------  -----  
23997 non-null  float64
dtypes: float64(1)
memory usage: 187.6 KB


In [101]:
df_prepared["carga_horaria_semanal"] = df_prepared["carga_horaria_semanal"].astype("Int64")

In [102]:
df_prepared["carga_horaria_semanal"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 23997 entries, 0 to 23996
Series name: carga_horaria_semanal
Non-Null Count  Dtype
--------------  -----
23997 non-null  Int64
dtypes: Int64(1)
memory usage: 211.0 KB


## Ajustes finais

- Renomeando colunas

In [115]:
df_prepared = df_prepared.rename(columns={
    "mes_referencia": "mes",
    "sexo_inferido": "sexo",
    "data_desligamento_formatada": "data_desligamento_",
    "tipo_pagamento_normalizado": "tipo_pagamento_"
})

- Aplicando filtro para pegar Prefeito e Vice Prefeito atuais

In [116]:
df_filtered = df_prepared[
    (~df_prepared['flag_agente_politico']) |
    (
        (df_prepared['flag_agente_politico']) &
        (df_prepared['data_admissao'] >= '2025-01-01')
    )
].copy()

- Selecionando colunas a serem exportadas

In [117]:
colunas_final = [
    "id_servidor",
    "sexo",
    "cargo",
    "categoria_cargo",
    "tipo_pagamento_",
    "proventos",
    "descontos",
    "liquido",
    "carga_horaria_semanal",
    "data_admissao",
    "data_desligamento_",
    "status_servidor",
    "mes"
]

- Validação rápida

In [118]:
print("df_prepared:", df_prepared.shape)
print("df_filtered:", df_filtered.shape)

df_prepared: (23997, 28)
df_filtered: (23995, 28)


- Garantir que políticos antigos não ficaram

In [119]:
df_filtered[
    (df_filtered["cargo"].str.contains("PREFEITO", case=False, na=False)) &
    (df_filtered["data_admissao"] < "2025-01-01")
]

,referencia,nome,cargo,data_admissao,tipo_regime,descontos,liquido,data_desligamento,proventos,contrato,...,tipo_pagamento_,nome_servidor_norm,sexo,id_servidor,cargo_norm,categoria_cargo,cargo_categorizado,flag_agente_politico,data_desligamento_,status_servidor


- Criação do df_final

In [120]:
df_final = df_filtered[colunas_final].copy()

- Tratamento da variável `mes`

In [121]:
def normalizar_mes(mes):
    if pd.isna(mes):
        return None

    mes = mes.strip().lower()
    mes = unicodedata.normalize("NFKD", mes).encode("ascii", "ignore").decode("utf-8")

    return mes[:3]

In [122]:
df_final["mes"] = df_final["mes"].apply(normalizar_mes)

- Ordenação categórica dos meses

In [123]:
ordem_meses = ["jan", "fev", "mar", "abr", "mai", "jun",
               "jul", "ago", "set", "out", "nov", "dez"]

df_final["mes"] = pd.Categorical(
    df_final["mes"],
    categories=ordem_meses,
    ordered=True
)

df_final = df_final.sort_values("mes").reset_index(drop=True)

- Renomeando coluna no df_final

In [124]:
df_final = df_final.rename(columns={
    "data_desligamento_": "data_desligamento",
    "tipo_pagamento_": "tipo_pagamento"
})

- Convertendo coluna `data_desligamento` para datetime

In [125]:
df_final["data_desligamento"] = pd.to_datetime(
    df_final["data_desligamento"],
    errors="coerce",
    dayfirst=True
)

- Exportação para EDA

In [126]:
output_path = "../data/processed/servidores2025.csv"

df_final.to_csv(
    output_path,
    index=False,
    encoding="utf-8"
)

## Considerações finais

Neste notebook foi concluída a etapa de **Data Preparation**, conforme a metodologia **CRISP-DM**.  
O foco esteve na transformação dos dados brutos em um conjunto de dados consistente, padronizado e pronto para análise.

Ao longo do processo, foram realizados os seguintes procedimentos principais:

- Padronização e tratamento de colunas numéricas (proventos, descontos e valores líquidos)
- Conversão e padronização de colunas de datas (admissão e desligamento)
- Inferência e consolidação da variável **sexo**
- Anonimização dos servidores por meio de identificadores únicos (`id_servidor`)
- Normalização e categorização dos cargos
- Avaliação e decisão consciente sobre colunas não utilizadas no momento
- Organização do dataset final com seleção e ordenação das colunas relevantes
- Preparação do DataFrame final (`df_final`) com ordenação temporal adequada

O resultado deste notebook é a geração de um **arquivo CSV estruturado**, salvo no diretório `data/processed`, que representa a base oficial para a próxima fase do projeto.

Este dataset foi construído com foco em:
- Reprodutibilidade
- Transparência metodológica
- Facilidade de análise
- Comunicação clara dos dados

A partir deste ponto, os dados estão prontos para serem utilizados em:
- Análises exploratórias
- Geração de indicadores
- Visualizações
- Comunicação pública dos resultados
- Construção de aplicações interativas (ex: Streamlit)

A próxima etapa do projeto será conduzida em um notebook dedicado exclusivamente à **Análise de Dados**, garantindo a separação clara entre preparação e interpretação dos dados.
